In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from Backtest import Backtest

In [3]:
import pandas as pd
from datetime import datetime
import os
from pathlib import Path
import logging
import yaml

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

def setup_directories():
    """Create necessary directories for the backtest system."""
    # Define base directories
    __file__ = 'main.ipynb'
    base_dir = os.path.abspath(os.path.dirname(__file__))
    data_dir = os.path.join(base_dir, 'data')
    db_dir = os.path.join(data_dir, 'db')
    cache_dir = os.path.join(data_dir, 'cache')
    
    # Create directories
    Path(data_dir).mkdir(exist_ok=True)
    Path(db_dir).mkdir(exist_ok=True)
    Path(cache_dir).mkdir(exist_ok=True)
    
    # Create default config if it doesn't exist
    config_path = os.path.join(base_dir, 'config.yaml')
    if not os.path.exists(config_path):
        default_config = {
            'cache': {
                'max_memory_cache_size': 1000,
                'cache_expiry_days': 1,
                'update_frequency': '1d',
                'compression_type': 'parquet'
            },
            'download': {
                'max_retries': 3,
                'retry_delay': 5,
                'batch_size': 100,
                'timeout': 30
            },
            'validation': {
                'min_data_points': 50,
                'max_missing_pct': 0.1,
                'price_threshold': 0.01
            }
        }
        with open(config_path, 'w') as f:
            yaml.dump(default_config, f)
    
    return {
        'base_dir': base_dir,
        'data_dir': data_dir,
        'db_dir': db_dir,
        'cache_dir': cache_dir,
        'config_path': config_path
    }

def test_backtest():
    """Test the Backtest class and its data fetching capabilities."""
    
    # Set up test parameters
    symbols = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'META']
    start_date = '2023-01-01'
    end_date = '2024-01-01'
    
    try:
        # Set up directories first
        dirs = setup_directories()
        
        logger.info("Testing with directories:")
        logger.info(f"Base dir: {dirs['base_dir']}")
        logger.info(f"DB path: {dirs['db_dir']}")
        logger.info(f"Cache dir: {dirs['cache_dir']}")
        
        # Initialize Backtest with proper config path
        backtest = Backtest(config_path=dirs['config_path'])
        
        # Fetch historical data
        logger.info(f"Fetching historical data for {len(symbols)} symbols...")
        historical_data = backtest.fetch_historical_data(symbols, start_date, end_date)

        print("\nData fetching test completed successfully!")
        print(historical_data)
        
        # Print summary statistics for each symbol
        for symbol, df in historical_data.items():
            print(f"\nSummary for {symbol}:")
            print(f"Date Range: {df.index.min()} to {df.index.max()}")
            print(f"Number of trading days: {len(df)}")
            print("\nPrice Statistics:")
            print(df['Close'].describe())
            print("\nVolume Statistics:")
            print(df['Volume'].describe())
            print("-" * 50)
        
        logger.info("Data fetching test completed successfully!")
        
        return historical_data
        
    except Exception as e:
        logger.error(f"Error during testing: {str(e)}")
        raise

if __name__ == "__main__":
    # For Jupyter notebook usage
    historical_data = test_backtest()

2024-12-18 14:17:21,706 - __main__ - INFO - Testing with directories:
2024-12-18 14:17:21,706 - __main__ - INFO - Base dir: /Users/calvinseamons/Repositories/NivlacSignals/CS470_Project/ML_Backtesting
2024-12-18 14:17:21,707 - __main__ - INFO - DB path: /Users/calvinseamons/Repositories/NivlacSignals/CS470_Project/ML_Backtesting/data/db
2024-12-18 14:17:21,707 - __main__ - INFO - Cache dir: /Users/calvinseamons/Repositories/NivlacSignals/CS470_Project/ML_Backtesting/data/cache
2024-12-18 14:17:21,708 - __main__ - INFO - Fetching historical data for 5 symbols...
2024-12-18 14:17:21,720 - __main__ - INFO - Data fetching test completed successfully!



Data fetching test completed successfully!
{}
